In [192]:
import torch
import torch.nn as nn
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel,AutoTokenizer, AutoModel
from tqdm import tqdm
import tensorflow as tf
import tensorflow_hub as hub
import os
import json
import pyarrow
import pandas as pd

In [193]:
import pandas as pd

data = pd.read_table('fra.txt', header= None)
data.rename(columns= {0: 'English', 1: 'French', 2: 'Citation'}, inplace= True)
data = data[:10000]
data

,English,French,Citation
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
9995,We've made it.,Nous avons réussi.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
9996,We've made it.,Nous y sommes parvenus.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
9997,We've matured.,Nous avons mûri.,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
9998,We've no time.,Nous n'avons pas le temps.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [194]:
pattern = r"[!'#$%&()*+,-./:;<=>?@[\]^`{|}~“”‘’«»‹›„‚–—…·•¡¿’\"\']"

eng_sent, french_sent = [], []

for e in range(len(data['English'])):
    eng_sent.append(re.sub(pattern, "", data['English'][e]))
    french_sent.append(re.sub(pattern, "", data['French'][e]))
#eng_sent[229801]

In [195]:
print(len(eng_sent))
print(len(french_sent))
def build_vocab(sentence_list):
  vocab_list = []
  for i in sentence_list:
    for j in i.split(' '):
      vocab_list.append(j)
  return vocab_list
eng_vocab = build_vocab(eng_sent)
fr_vocab = build_vocab(french_sent)
len(eng_vocab)

10000
10000


26262

In [34]:
START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''
eng_vocab.insert(0,START_TOKEN)
eng_vocab.append(PADDING_TOKEN)
eng_vocab.append(END_TOKEN)
fr_vocab.insert(0,START_TOKEN)
fr_vocab.append(PADDING_TOKEN)
fr_vocab.append(END_TOKEN)

In [35]:
index_to_french = {k:v for k,v in enumerate(fr_vocab)}
french_to_index = {v:k for k,v in enumerate(fr_vocab)}
index_to_english = {k:v for k,v in enumerate(eng_vocab)}
english_to_index = {v:k for k,v in enumerate(eng_vocab)}
english_to_index['ahead']
eng_sent[:4]
print(len(english_to_index))
print(len(fr_vocab))

2261
31938


In [71]:
french_to_index['Souriez\u2009']

140

# writing the french embeddings for time saving

In [181]:
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F

def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled = scaled.permute(1, 0, 2, 3) + mask
        scaled = scaled.permute(1, 0, 2, 3)
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length)
                          .reshape(self.max_sequence_length, 1))
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

class SentenceEmbedding(nn.Module):
    "For a given sentence, create an embedding"
    def __init__(self, max_sequence_length, d_model, language_to_index, vocab_size, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = vocab_size
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN

    def batch_tokenize(self, batch, start_token, end_token):

        def tokenize(sentence, start_token, end_token):
            sentence_word_indicies = [self.language_to_index[token] for token in sentence.split(' ')]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
           tokenized.append( tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(get_device())

    def forward(self, x, start_token, end_token): # sentence
        x = self.batch_tokenize(x, start_token, end_token)
        print('---------tokenization done --------')
        x = self.embedding(x)
        print('---------Sentence Embedding done --------')
        pos = self.position_encoder().to(get_device())
        print('---------Positional Encoding done --------')
        x = self.dropout(x + pos)
        return x


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask):
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        out = self.linear_layer(values)
        return out


class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x


class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, self_attention_mask):
        #print('------------ENCODER ACTIVATED----------')
        residual_x = x.clone()
        x = self.attention(x, mask=self_attention_mask)
        #print('-----------Attention Done---------')
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        #print('-----------LayerNorm1 Done---------')
        residual_x = x.clone()
        x = self.ffn(x)
        #print('-----------FFNN Done---------')
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        #print('-----------LayerNorm2 Done---------')
        return x

class SequentialEncoder(nn.Sequential):
    def forward(self, *inputs):
        x, self_attention_mask  = inputs
        for module in self._modules.values():
            x = module(x, self_attention_mask)
        return x

class Encoder(nn.Module):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 vocab_size,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index,vocab_size, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialEncoder(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])

    def forward(self, x, self_attention_mask, start_token, end_token):
        x = self.sentence_embedding(x, start_token, end_token)
        print('DATA PREPROCESSING DONE')
        x = self.layers(x, self_attention_mask)
        return x


class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model)
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, y, mask):
        batch_size, sequence_length, d_model = x.size() # in practice, this is the same for both languages...so we can technically combine with normal attention
        kv = self.kv_layer(x)
        q = self.q_layer(y)
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)
        kv = kv.permute(0, 2, 1, 3)
        q = q.permute(0, 2, 1, 3)
        k, v = kv.chunk(2, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask) # We don't need the mask for cross attention, removing in outer function!
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, d_model)
        out = self.linear_layer(values)
        return out


class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.layer_norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y.clone()
        y = self.self_attention(y, mask=self_attention_mask)
        #print('---------ATTENTION DONE----------')
        y = self.dropout1(y)
        y = self.layer_norm1(y + _y)
       # print('---------layernorm1 DONE----------')
        _y = y.clone()
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
        #print('--------- CROSS ATTENTION DONE----------')
        y = self.dropout2(y)
        y = self.layer_norm2(y + _y)
        #print('---------LayerNorm2 DONE----------')
        _y = y.clone()
        y = self.ffn(y)
        #print('---------FFNN DONE----------')
        y = self.dropout3(y)
        y = self.layer_norm3(y + _y)
       #print('---------LayerNorm3 DONE----------')
        return y


class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for module in self._modules.values():
            y = module(x, y, self_attention_mask, cross_attention_mask)
        return y

class Decoder(nn.Module):
    def __init__(self,
                 d_model,
                 ffn_hidden,
                 num_heads,
                 drop_prob,
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 vocab_size,
                 START_TOKEN,
                 END_TOKEN,
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index,vocab_size,START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def forward(self, x, y, self_attention_mask, cross_attention_mask, start_token, end_token):
        y = self.sentence_embedding(y, start_token, end_token)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y


class Transformer(nn.Module):
    def __init__(self,
                d_model,
                ffn_hidden,
                num_heads,
                drop_prob,
                num_layers,
                max_sequence_length,
                en_vocab_size,
                fr_vocab_size,
                english_to_index,
                french_to_index,
                START_TOKEN,
                END_TOKEN,
                PADDING_TOKEN
                ):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, english_to_index,en_vocab_size, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, french_to_index,fr_vocab_size, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = nn.Linear(d_model, fr_vocab_size)
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    def forward(self,
                x,
                y,
                encoder_self_attention_mask=None,
                decoder_self_attention_mask=None,
                decoder_cross_attention_mask=None,
                enc_start_token=True,
                enc_end_token=True,
                dec_start_token=True, # We should make this true
                dec_end_token=True): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        print('ENCODER COMPLETED')
        print('DECODER ACTIVATED')
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        print('DECODER completed')
        out = self.linear(out)
        return out

In [182]:
d_model = 512
batch_size = 1
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 5
max_sequence_length = 150
fr_vocab_size = len(fr_vocab)
eng_vocab_size = len(eng_vocab)
transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          eng_vocab_size,
                          fr_vocab_size,
                          english_to_index,
                          french_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [183]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, french_sentences):
        self.english_sentences = english_sentences
        self.french_sentences = french_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.french_sentences[idx]

In [184]:
dataset = TextDataset(eng_sent[:1000], french_sent[:1000])


In [185]:
len(dataset)

1000

In [186]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [187]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=french_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [188]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, fr_sentence_length = len(eng_batch[idx]), len(fr_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      fr_chars_to_padding_mask = np.arange(fr_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, fr_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, fr_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, fr_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [189]:
for batch_num, batch in enumerate(iterator):
    print(batch)

[('Go',), ('Va ',)]
[('Go',), ('Marche',)]
[('Go',), ('En route ',)]
[('Go',), ('Bouge ',)]
[('Hi',), ('Salut ',)]
[('Hi',), ('Salut',)]
[('Run',), ('Cours\u202f',)]
[('Run',), ('Courez\u202f',)]
[('Run',), ('Prenez vos jambes à vos cous ',)]
[('Run',), ('File ',)]
[('Run',), ('Filez ',)]
[('Run',), ('Cours ',)]
[('Run',), ('Fuyez ',)]
[('Run',), ('Fuyons ',)]
[('Run',), ('Cours\u202f',)]
[('Run',), ('Courez\u202f',)]
[('Run',), ('Prenez vos jambes à vos cous ',)]
[('Run',), ('File ',)]
[('Run',), ('Filez ',)]
[('Run',), ('Cours ',)]
[('Run',), ('Fuyez ',)]
[('Run',), ('Fuyons ',)]
[('Who',), ('Qui ',)]
[('Wow',), ('Ça alors\u202f',)]
[('Wow',), ('Waouh\xa0',)]
[('Wow',), ('Wah\xa0',)]
[('Duck',), ('À terre\xa0',)]
[('Duck',), ('Baissetoi\xa0',)]
[('Duck',), ('Baissezvous\xa0',)]
[('Fire',), ('Au feu ',)]
[('Help',), ('À laide\u202f',)]
[('Hide',), ('Cachetoi',)]
[('Hide',), ('Cachezvous',)]
[('Jump',), ('Saute',)]
[('Jump',), ('Saute',)]
[('Stop',), ('Ça suffit\u202f',)]
[('Stop',), (

In [190]:
fr_sentence = ("",)
eng_sentence = ("hi",)
encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, fr_sentence)
predictions = transformer(eng_sentence,
                          fr_sentence,
                          encoder_self_attention_mask.to(device),
                          decoder_self_attention_mask.to(device),
                          decoder_cross_attention_mask.to(device),
                          enc_start_token=False,
                          enc_end_token=False,
                          dec_start_token=True,
                          dec_end_token=False)
next_token_prob_distribution = predictions[0] # not actual probs
next_token_index = torch.argmax(next_token_prob_distribution,axis=1)
for idx in next_token_index:
      if idx == french_to_index[END_TOKEN]:
        break
      fr_sentence = (fr_sentence[0] + index_to_french[idx.item()], )
print(f"Evaluation translation (hi) : {fr_sentence}")
print("-------------------------------------------")

---------tokenization done --------
---------Sentence Embedding done --------
---------Positional Encoding done --------
DATA PREPROCESSING DONE
ENCODER COMPLETED
DECODER ACTIVATED
---------tokenization done --------
---------Sentence Embedding done --------
---------Positional Encoding done --------
DECODER completed
Evaluation translation (hi) : ('TompartirlaavonsBonjourviandeJepartirJeDétendezvous\u202fIlpartiraJeJeaResteaIlDétendezvous\u202favonslaàmangealentementaaaResteaDétendezvous\u202fJeaDétendezvous\u202fBonjourIlcœuraJemeagardesaJeIldûJeBonjourstableJepartirmeJeJemeallerBonjournouveau\u202fJeJeavonsDétendezvous\u202fJeJedûJeDétendezvous\u202fBonjourdûneJeBonjourBonjourfautSoismeBonjourDétendezvous\u202fpiquéSoisBonjourlentementmeJeJemeJeDétendezvous\u202fDétendezvous\u202fJeDétendezvous\u202fmegardesmePrendsyDétendezvous\u202fBonjourDétendezvous\u202fapayéJeDétendezvous\u202fJemeJeIlJeIlJemangeaaJeDétendezvous\u202fmemesuismeDétendezvous\u202fIlpayéapayéDétendezvous\u202fpay

In [191]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 20

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, fr_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, fr_batch)
        optim.zero_grad()
        fr_predictions = transformer(eng_batch,
                                     fr_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(fr_batch, start_token=False, end_token=True)
        loss = criterian(
            fr_predictions.view(-1, fr_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == french_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % len(batch) == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"french Translation: {fr_batch[0]}")
            fr_sentence_predicted = torch.argmax(fr_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in fr_sentence_predicted:
              if idx == french_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_french[idx.item()]
            print(f"french Prediction: {predicted_sentence}")

            transformer.eval()
            fr_sentence = ("",)
            eng_sentence = ("hi",)
            encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, fr_sentence)
            predictions = transformer(eng_sentence,
                                      fr_sentence,
                                      encoder_self_attention_mask.to(device),
                                      decoder_self_attention_mask.to(device),
                                      decoder_cross_attention_mask.to(device),
                                      enc_start_token=False,
                                      enc_end_token=False,
                                      dec_start_token=True,
                                      dec_end_token=False)
            next_token_prob_distribution = predictions[0] # not actual probs
            next_token_index = torch.argmax(next_token_prob_distribution,axis=1)
            for idx in next_token_index:
                  if idx == french_to_index[END_TOKEN]:
                    break
                  fr_sentence = (fr_sentence[0] + index_to_french[idx.item()], )
            print(f"Evaluation translation (hi) : {fr_sentence}")

Epoch 0
---------tokenization done --------
---------Sentence Embedding done --------
---------Positional Encoding done --------
DATA PREPROCESSING DONE
ENCODER COMPLETED
DECODER ACTIVATED
---------tokenization done --------
---------Sentence Embedding done --------
---------Positional Encoding done --------
DECODER completed
Iteration 0 : 10.47547435760498
English: Go
french Translation: Va 
french Prediction: nousBonjourconcentréeaDétendezvous IlIlJeBonjourmariéEnSoislemeDétendezvous payésuisBonjourstableJepartirsuisyResteàIlavonsfautlefaisaisJeydûJenedûJeSoisJeàaamestableJesourisDétendezvous JeDétendezvous dûdûenadûaavonsJeJeaDétendezvous EllesResteàDétendezvous JeJeJeàResteDétendezvous JaimememeaDétendezvous medûNoussuisaIlBonjourDétendezvous JeDétendezvous Détendezvous ymeJeDétendezvous JeameyDétendezvous IlenmeJemeJeEnmeResteavezmefauteJeDétendezvous yJestablenemeDétendezvous meJeallermeàJeJedûBonjourResteJeJeoùpartirpayéDétendezvous Détendezvous IlpayéstableDétendezvous Détendez

KeyboardInterrupt: ignored